In [ ]:
# !pip install kfp==1.8.18
# # !pip install kubeflow-katib==0.13.0

In [211]:
# Copyright 2019 The Kubeflow Authors
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import kfp
import kfp.dsl as dsl
from kfp import components
import kubernetes as k8s
from kfp import compiler, dsl
from kubernetes.client import V1VolumeMount
import pprint
import numpy as np


@dsl.pipeline(
    name="blueberry detection",
    description="blueberry detection pipeline"
)
def full_pipeline(size: str="2Gi"):
    
    vop = dsl.PipelineVolume(
        pvc="biosens-volume-main", # change with the corresponding volume name #
        name="biosens-volume-main",
    )

    download_component = kfp.components.load_component_from_file('download_model.yaml')
    train_model = kfp.components.load_component_from_file('train_model.yaml')
    postprocessing_model = kfp.components.load_component_from_file('postprocessing_model.yaml')

    
    download_component_task = download_component(
        "blueberry-full", # minIO bucket where the data is storade
        "/mnt"
        ).add_pvolumes({"/mnt": vop})
    download_component_task.execution_options.caching_strategy.max_cache_staleness = "P0D" # avoiding caching
    train_model_task = train_model(
        [1e-3], #'Learning rate'
        [1], #'Lambda parametar'
        [5], #'Stepovi arr'
        [8], #'num_epochs'
        ['bce'], #'Lloss_type'
        [2], #'Batch_size'
        "UNet++", #'Type of Network that will be run. Available model architectures: UNet3, UNet++, SegNet, PSPNet, UperNet, DUC_HDCNet'
        "cuda", #'will torch run on cuda (gpu) or cpu'
        "/mnt/FullSet/trening_set/img/", # 'trening_location'
        "/mnt/FullSet/validation_set/img/", #'validation_location'
        "/mnt/FullSet/test_set/img/", #'test_location'
        # "/mnt" # if download is pruned this should replace the line bellow
        download_component_task.output
        ).add_pvolumes({"/mnt": vop})

    train_model_task.set_gpu_limit(1)
    train_model_task.execution_options.caching_strategy.max_cache_staleness = "P0D" # avoiding caching
    postprocessing_model = postprocessing_model(
        "/mnt/logs/train_pipeline_nettype/", # nettype can be changed with corresponding net type in the moment of creating pipelines
        "blueberry-results", # output minIO bucket for storing results
        train_model_task.output
        ).add_pvolumes({"/mnt": vop})
    
    
    
    print("gotovo sve")
if __name__ == '__main__':
    file_name = "full_nettype_pipeline.yaml" # nettype can be changed with corresponding net type in the moment of creating pipelines
    kfp.compiler.Compiler().compile(full_pipeline, file_name)

gotovo sve


In [212]:
import kfp
import random
client=kfp.Client()
filepath = '/home/jovyan/biosens-volume-main/' + file_name
name = 'train_pipeline_nettype_timestamp' # nettype and timestamp can be changed with corresponding net type and time in the moment of creating pipelines
print("Uploaded pipeline:", name)
pipeline = client.pipeline_uploads.upload_pipeline(filepath, name=name)

Uploaded pipeline: TrainPipelineNetType_DUC_HDCNet_0910_20112023
